In [1]:
# import project.avi video and perform power spectrum analysis on it

import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
from scipy import fftpack

# read video
cap = cv2.VideoCapture('project.avi')

# get video properties
fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# create empty list to store frames
frames = []

# read frames
for i in range(frame_count):
    ret, frame = cap.read()
    if ret:
        frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY))
    else:
        break
    
# convert list to numpy array
frames = np.array(frames)


# save power spectrum


In [2]:
frames.shape

# swap axes 2 and 1 to get (frame, x, y) shape
frames = np.swapaxes(frames, 1, 2)

frames.shape

(3600, 1280, 720)

In [8]:
def frames_to_text(frames, filename):
    num_frames, height, width = frames.shape
    with open(filename, 'w') as f:
        f.write("frameset text\n")
        f.write(f"{num_frames} 1 {width} {height}\n")
        for frame in frames:
            for row in frame:
                f.write(' '.join(map(str, row)) + '\n')

# Use the function
frames_2_save = frames[:600, :, :]
frames_to_text(frames_2_save, 'output.txt')

In [9]:
import numpy as np

def text_to_frames(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()
    num_frames, _, width, height = map(int, lines[1].split())
    frames = np.zeros((num_frames, height, width))
    pixel_values = map(float, ' '.join(lines[2:]).split())
    for frame in range(num_frames):
        for y in range(height):
            for x in range(width):
                frames[frame, y, x] = next(pixel_values)
    return frames

# Use the function
new_frames = text_to_frames('output.txt')

In [39]:
def plot_power_spectrum(data, pixels_per_degree, fps, cmap='viridis'):
    time_steps, y_pixels, x_pixels = data.shape

    # Perform 3D FFT (time, y, x)
    fft_data = np.fft.fftn(data, axes=(0, 1, 2))
    # Compute the power spectrum
    power_spectrum = np.abs(fft_data)**2
    # Log transform the power spectrum
    log_power_spectrum = np.log(power_spectrum + 1e-10)  # Adding a small constant to avoid log(0)
    # Extract one frame of the original stimulus
    frame = data[data.shape[0]//2, :, :]
    # Compute the 2D spatial frequency distribution (SFx vs SFy)
    spatial_spectrum = np.mean(log_power_spectrum, axis=0)
    # Compute the 2D distribution of SFy vs TF and SFx vs TF
    sfy_vs_tf = np.mean(log_power_spectrum, axis=2)
    sfx_vs_tf = np.mean(log_power_spectrum, axis=1)
    # Convert pixel frequencies to cycles per degree
    spatial_freq_y = np.fft.fftfreq(y_pixels) / pixels_per_degree
    spatial_freq_x = np.fft.fftfreq(x_pixels) / pixels_per_degree
    # Convert temporal frequencies to Hz (frames per second)
    freq_time = np.fft.fftfreq(time_steps, d=1/fps)
    
    # Plotting
    fig, axes = plt.subplots(2, 2, figsize=(10, 10))
    # Plot one frame of the original stimulus
    axes[0, 0].imshow(frame, cmap='gray')
    axes[0, 0].set_title('One Frame of the Stimulus')

    # Plot the spatial distribution of the log power spectrum (SFx vs SFy)
    im = axes[0, 1].imshow(np.fft.fftshift(spatial_spectrum), cmap='gray', 
                        extent=[spatial_freq_x.min(), spatial_freq_x.max(), spatial_freq_y.min(), spatial_freq_y.max()])
    axes[0, 1].set_title('Spatial Log Power Spectrum (SFx vs SFy)')
    axes[0, 1].set_xlabel('Spatial Frequency (cycles/degree)')
    axes[0, 1].set_ylabel('Spatial Frequency (cycles/degree)')
    axes[0, 1].set_xlim([spatial_freq_x.min(), spatial_freq_x.max()])
    axes[0, 1].set_ylim([spatial_freq_y.min(), spatial_freq_y.max()])
    # axes[0, 1].set_xlim([-0.03, 0.03])
    # axes[0, 1].set_ylim([-0.03, 0.03])

    # Plot SFy vs TF
    im = axes[1, 0].imshow(np.fft.fftshift(sfy_vs_tf), cmap='gray', aspect='auto', 
                        extent=[freq_time.min(), freq_time.max(), spatial_freq_y.min(), spatial_freq_y.max()])
    axes[1, 0].set_title('SFy vs TF')
    axes[1, 0].set_xlabel('Temporal Frequency (Hz)')
    axes[1, 0].set_ylabel('Spatial Frequency (cycles/degree)')
    axes[1, 0].set_xlim([freq_time.min(), freq_time.max()])
    axes[1, 0].set_ylim([spatial_freq_y.min(), spatial_freq_y.max()])
    axes[1, 0].set_xlim([-30, 30])
    # axes[1, 0].set_ylim([-0.03, 0.03])

    # Plot SFx vs TF
    im = axes[1, 1].imshow(np.fft.fftshift(sfx_vs_tf), cmap='gray', aspect='auto', 
                        extent=[freq_time.min(), freq_time.max(), spatial_freq_x.min(), spatial_freq_x.max()])
    axes[1, 1].set_title('SFx vs TF')
    axes[1, 1].set_xlabel('Temporal Frequency (Hz)')
    axes[1, 1].set_ylabel('Spatial Frequency (cycles/degree)')
    axes[1, 1].set_xlim([freq_time.min(), freq_time.max()])
    axes[1, 1].set_ylim([spatial_freq_x.min(), spatial_freq_x.max()])
    axes[1, 1].set_xlim([-30, 30])
    # axes[1, 1].set_ylim([-0.03, 0.03])

    plt.tight_layout()
    plt.show()

    # return the power spectrum and figure
    return power_spectrum, fig


data = frames[:600, :600, :600]
# Use the function
ps1, fig1 = plot_power_spectrum(data, pixels_per_degree=17, fps=60)
ps2, fig2 = plot_power_spectrum(data, pixels_per_degree=34, fps=60)